<a href="https://colab.research.google.com/github/saifulislamr16/bangla_fake_news/blob/main/Capstone_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install flask pyngrok selenium tensorflow nltk

# Model Create and save

In [ ]:
# Install necessary libraries
!pip install numpy pandas matplotlib seaborn tensorflow imbalanced-learn nltk selenium

# Install ChromeDriver for Selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import RandomOverSampler
import nltk
from nltk.corpus import stopwords
import string
import re
import pickle

# Load datasets
authentic_df = pd.read_csv('/content/drive/MyDrive/extracted/Authentic-48K.csv')
fake_df = pd.read_csv('/content/drive/MyDrive/extracted/Fake-1K.csv')

# Combine datasets
authentic_df['label'] = 0
fake_df['label'] = 1
new_df = pd.concat([authentic_df, fake_df], ignore_index=True, axis=0)
new_df.drop(['articleID', 'domain', 'date', 'category'], inplace=True, axis=1)
new_df['news'] = new_df['headline'] + ' ' + new_df['content']
new_df = new_df.drop(['headline', 'content'], axis=1)
new_df['label'] = new_df['label'].astype(int)

# Plot label distribution before balancing
sns.countplot(data=new_df, x=new_df['label'])
plt.title("Label Distribution Before Balancing")
plt.show()

# Separate features and labels
X = new_df['news']
y = new_df['label']

# Handle class imbalance using RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X.values.reshape(-1, 1), y)
df_oversampled = pd.DataFrame({'news': X_over.flatten(), 'label': y_over})
df_oversampled_shuffled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Plot label distribution after balancing
sns.countplot(data=df_oversampled_shuffled, x=df_oversampled_shuffled['label'])
plt.title("Label Distribution After Balancing")
plt.show()

# Preprocess text
nltk.download('stopwords')
stop_words = set(stopwords.words('bengali'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text

df_oversampled_shuffled['news'] = df_oversampled_shuffled['news'].apply(preprocess)

# Split data
x_train, x_test, y_train, y_test = train_test_split(df_oversampled_shuffled['news'], df_oversampled_shuffled['label'], test_size=0.1, random_state=0, stratify=df_oversampled_shuffled['label'])

# Tokenize and pad sequences
tk = Tokenizer()
tk.fit_on_texts(x_train)
seq_tr = tk.texts_to_sequences(x_train)
vec_tr = pad_sequences(seq_tr, padding='post', maxlen=200)
seq_test = tk.texts_to_sequences(x_test)
vec_test = pad_sequences(seq_test, padding='post', maxlen=200)

train_x = np.array(vec_tr)
test_x = np.array(vec_test)
train_y = np.array(y_train)
test_y = np.array(y_test)

# Build the model with increased regularization and dropout
def build_model(input_dim):
    model = tf.keras.models.Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=200))
    model.add(Bidirectional(LSTM(4, return_sequences=True, kernel_regularizer=l2(0.2))))
    model.add(BatchNormalization())
    model.add(Dropout(0.7))
    model.add(Bidirectional(LSTM(2, kernel_regularizer=l2(0.2))))
    model.add(BatchNormalization())
    model.add(Dropout(0.7))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.2)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Cross-validation setup
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

histories = []
models = []

for train_index, val_index in kf.split(train_x, train_y):
    X_train_fold, X_val_fold = train_x[train_index], train_x[val_index]
    y_train_fold, y_val_fold = train_y[train_index], train_y[val_index]

    model = build_model(input_dim=len(tk.word_index) + 1)

    # Early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-6, verbose=1)

    history = model.fit(X_train_fold, y_train_fold, epochs=4, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping, lr_scheduler])
    histories.append(history)
    models.append(model)

# Select the best model based on validation loss
best_model_index = np.argmin([min(history.history['val_loss']) for history in histories])
best_model = models[best_model_index]

# Save the best model
best_model_path = '/content/drive/MyDrive/extracted/bangla_fake_news_model.h5'
best_model.save(best_model_path)

# Save the tokenizer
tokenizer_path = '/content/drive/MyDrive/extracted/tokenizer.pkl'
with open(tokenizer_path, 'wb') as file:
    pickle.dump(tk, file)

# Plot accuracy and loss
history = histories[best_model_index]

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Epoch vs Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epoch vs Loss')
plt.legend()
plt.show()

# Evaluate the best model
y_pred = (best_model.predict(test_x) > 0.5).astype("int32")
cm = confusion_matrix(test_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Authentic', 'Fake'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# Print classification report
print(classification_report(test_y, y_pred, target_names=['Authentic', 'Fake']))

# Function to get news content from a URL using Selenium
def get_news_content_selenium(url):
    try:
        # Configure Selenium WebDriver
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=chrome_options)

        driver.get(url)
        # You may need to wait for specific elements to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

        # Use the updated method to find elements
        content = driver.find_element(By.TAG_NAME, 'body').text
        driver.quit()  # It's important to close the driver after the task is done
        return content
    except Exception as e:
        return f"Error: Could not retrieve content. Exception: {e}"

# Function to predict news authenticity from content
def predict_news(news, model, tokenizer):
    news = preprocess(news)
    seq = tokenizer.texts_to_sequences([news])
    padded = pad_sequences(seq, padding='post', maxlen=200)
    prediction = model.predict(padded)[0][0]
    if prediction > 0.5:
        return "Fake News"
    else:
        return "Authentic News"

# Function to predict news authenticity from a URL
def predict_news_url_selenium(url, model, tokenizer):
    # Fetch the content from the URL
    content = get_news_content_selenium(url)

    # Check for errors in fetching content
    if content.startswith("Error"):
        return content

    print(f"News content from URL:\n{content}\n")

    # Predict the authenticity of the fetched content
    return predict_news(content, model, tokenizer)

# Interactive function to choose between direct news input or URL input
def interactive_prediction(model, tokenizer):
    choice = input("Enter '1' to input news text directly or '2' to input a news URL: ")

    if choice == '1':
        news = input("Enter news text: ")
        result = predict_news(news, model, tokenizer)
    elif choice == '2':
        url = input("Enter news URL: ")
        result = predict_news_url_selenium(url, model, tokenizer)
    else:
        result = "Invalid choice. Please enter '1' or '2'."

    print(f'The news is predicted to be: {result}')

# Load the model and tokenizer
loaded_model = tf.keras.models.load_model(best_model_path)
with open(tokenizer_path, 'rb') as file:
    loaded_tokenizer = pickle.load(file)

# Call the interactive prediction function
interactive_prediction(loaded_model, loaded_tokenizer)

# Output code to test

In [ ]:
# Import libraries
import os
import re
import string
import numpy as np
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
import nltk
import pickle

nltk.download('stopwords')

# Flask app setup
app = Flask(__name__)

# Ensure the templates folder exists
os.makedirs('templates', exist_ok=True)

# Save the enhanced HTML template with improved responsive design
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bangla Fake News Detector</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(135deg, #1a1a2e, #16213e);
            color: #fff;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            overflow: hidden;
            position: relative;
        }

        /* Animated background with particles */
        .background-animation {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: radial-gradient(circle, rgba(0, 212, 255, 0.1), transparent);
            overflow: hidden;
            z-index: 0;
        }

        .particle {
            position: absolute;
            background-color: rgba(0, 212, 255, 0.7);
            width: 10px;
            height: 10px;
            border-radius: 50%;
            opacity: 0;
            animation: moveParticle 10s linear infinite;
            z-index: 0;
        }

        @keyframes moveParticle {
            0% {
                transform: translateY(100vh) translateX(0);
                opacity: 1;
            }
            100% {
                transform: translateY(-100vh) translateX(calc(100vw * var(--direction)));
                opacity: 0;
            }
        }

        .container {
            background: rgba(22, 33, 62, 0.9);
            padding: 40px;
            border-radius: 12px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
            width: 100%;
            max-width: 500px;
            text-align: center;
            position: relative;
            overflow: hidden;
            z-index: 1;
        }

        .container::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: linear-gradient(135deg, #00d4ff, #28a745, #dc3545);
            background-size: 200% 200%;
            opacity: 0.05;
            z-index: -1;
            animation: gradientMove 15s infinite;
        }

        @keyframes gradientMove {
            0% {
                background-position: 0% 50%;
            }
            50% {
                background-position: 100% 50%;
            }
            100% {
                background-position: 0% 50%;
            }
        }

        h1 {
            color: #00d4ff;
            margin-bottom: 20px;
            font-size: 2em;
            animation: fadeInDown 1s ease-out;
        }

        label {
            display: block;
            font-weight: 500;
            margin-bottom: 10px;
            text-align: left;
            animation: fadeInUp 1s ease-out;
        }

        textarea, input[type="submit"], input[type="radio"] {
            width: 100%;
            padding: 12px;
            margin: 10px 0;
            border: none;
            border-radius: 8px;
            box-sizing: border-box;
            font-size: 16px;
            background: rgba(0, 0, 0, 0.2);
            color: #fff;
            z-index: 2;
            position: relative;
        }

        input[type="radio"] {
            width: auto;
            margin-right: 10px;
        }

        input[type="submit"] {
            background: linear-gradient(135deg, #00d4ff, #00a3cc);
            color: #fff;
            cursor: pointer;
            transition: background 0.3s;
            position: relative;
            z-index: 2;
            overflow: hidden;
        }

        input[type="submit"]::before {
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 300%;
            height: 100%;
            background: rgba(255, 255, 255, 0.2);
            transition: left 0.3s ease-out;
            z-index: -1;
        }

        input[type="submit"]:hover::before {
            left: 0;
        }

        input[type="submit"]:hover {
            background: linear-gradient(135deg, #00a3cc, #007a99);
        }

        .result {
            margin-top: 20px;
            padding: 15px;
            border-radius: 8px;
            font-size: 1.2em;
            display: none;
            z-index: 2;
            position: relative;
            animation: slideIn 0.5s ease-out;
        }

        .result.authentic {
            background-color: rgba(40, 167, 69, 0.9);
            color: #fff;
        }

        .result.fake {
            background-color: rgba(220, 53, 69, 0.9);
            color: #fff;
        }

        .loader {
            border: 8px solid rgba(243, 243, 243, 0.3);
            border-top: 8px solid #00d4ff;
            border-radius: 50%;
            width: 60px;
            height: 60px;
            animation: spin 2s linear infinite;
            margin: 20px auto;
            display: none;
            z-index: 2;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .progress-container {
            width: 100%;
            background-color: #e0e0e0;
            border-radius: 25px;
            margin-top: 20px;
            overflow: hidden;
            z-index: 2;
        }

        .progress-bar {
            height: 20px;
            width: 0;
            background-color: #00d4ff;
            border-radius: 25px;
            transition: width 0.5s;
            z-index: 2;
        }

        @keyframes fadeInDown {
            from {
                opacity: 0;
                transform: translateY(-20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateX(-20px);
            }
            to {
                opacity: 1;
                transform: translateX(0);
            }
        }
    </style>
</head>
<body>
    <div class="background-animation">
        <!-- Adding particles for animation -->
        <div class="particle" style="--direction: 0.5; animation-delay: 0s;"></div>
        <div class="particle" style="--direction: -0.5; animation-delay: 1s;"></div>
        <div class="particle" style="--direction: 0.7; animation-delay: 2s;"></div>
        <div class="particle" style="--direction: -0.7; animation-delay: 3s;"></div>
        <div class="particle" style="--direction: 0.3; animation-delay: 4s;"></div>
        <div class="particle" style="--direction: -0.3; animation-delay: 5s;"></div>
    </div>
    <div class="container">
        <h1>Bangla Fake News Detector</h1>
        <form id="newsForm">
            <label for="data_type">Choose input type:</label>
            <input type="radio" id="text" name="data_type" value="text" checked>
            <label for="text" style="display: inline;">News Text</label>
            <input type="radio" id="url" name="data_type" value="url">
            <label for="url" style="display: inline;">News URL</label>
            <label for="news">Enter news text or URL:</label>
            <textarea id="news" name="news" rows="4"></textarea>
            <input type="submit" value="Submit">
        </form>
        <div class="loader" id="loader"></div>
        <div id="result" class="result"></div>
        <div class="progress-container">
            <div id="progressBar" class="progress-bar"></div>
        </div>
    </div>
    <script>
        document.addEventListener('DOMContentLoaded', function () {
            const form = document.getElementById('newsForm');
            const resultDiv = document.getElementById('result');
            const loader = document.getElementById('loader');
            const progressBar = document.getElementById('progressBar');

            form.onsubmit = async function (e) {
                e.preventDefault();
                resultDiv.style.display = 'none';
                loader.style.display = 'block';
                progressBar.style.width = '0';
                const formData = new FormData(form);
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
                const data = await response.json();
                loader.style.display = 'none';
                resultDiv.style.display = 'block';
                resultDiv.innerHTML = `Prediction: ${data.prediction} (${data.percentage}%)`;
                if (data.prediction === 'Authentic News') {
                    resultDiv.className = 'result authentic';
                    progressBar.style.backgroundColor = '#28a745';
                } else {
                    resultDiv.className = 'result fake';
                    progressBar.style.backgroundColor = '#dc3545';
                }
                progressBar.style.width = `${data.percentage}%`;
            };
        });
    </script>
</body>
</html>

"""
with open('templates/index.html', 'w') as f:
    f.write(html_template)

# Load the model and tokenizer
model_path = '/content/drive/MyDrive/Capstone_shared/bangla_fake_news_model.h5'
tokenizer_path = '/content/drive/MyDrive/Capstone_shared/tokenizer.pkl'

# Load the model using Keras' load_model method
model = tf.keras.models.load_model(model_path)

# Load the tokenizer
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Preprocessing function
stop_words = set(stopwords.words('bengali'))
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Function to predict news authenticity
def predict_news(news):
    news = preprocess(news)
    seq = tokenizer.texts_to_sequences([news])
    padded = pad_sequences(seq, padding='post', maxlen=200)
    prediction = model.predict(padded)[0][0]
    percentage = round(prediction * 100, 2) if prediction > 0.5 else round((1 - prediction) * 100, 2)
    return ("Fake News" if prediction > 0.5 else "Authentic News"), percentage

# Function to get news content from a URL using Selenium
def get_news_content_selenium(url):
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=chrome_options)

        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        content = driver.find_element(By.TAG_NAME, 'body').text
        driver.quit()
        return content
    except Exception as e:
        return f"Error: Could not retrieve content. Exception: {e}"

# Function to predict news authenticity from a URL
def predict_news_url_selenium(url):
    if isinstance(url, str):
        content = get_news_content_selenium(url)
        if content.startswith("Error"):
            return content, 0
        return predict_news(content)
    else:
        return "Error: URL is not a string.", 0

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if request.method == 'POST':
            data_type = request.form.get('data_type')
            if data_type == 'text':
                news = request.form.get('news')
                prediction, percentage = predict_news(news)
            elif data_type == 'url':
                url = request.form.get('news')
                prediction, percentage = predict_news_url_selenium(url)
            return jsonify({'prediction': prediction, 'percentage': percentage})
    except Exception as e:
        return str(e), 400

# Set ngrok auth token and connect
ngrok.set_auth_token("2iq8btcKUJUwcpIvds0cZ3VcqNh_hZoSVsfv93hx5NFpAGxq")  # Ensure to set your ngrok auth token here
public_url = ngrok.connect(5000, bind_tls=True)
print('Public URL:', public_url)

# Run the Flask app
app.run(port=5000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Public URL: NgrokTunnel: "https://b8d3-34-125-44-120.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 05:31:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 05:31:10] "GET /favicon.ico HTTP/1.1" 404 -


# Another style

In [ ]:
# Import libraries
import os
import re
import string
import numpy as np
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
import nltk
import pickle

nltk.download('stopwords')

# Flask app setup
app = Flask(__name__)

# Ensure the templates folder exists
os.makedirs('templates', exist_ok=True)

# Save the enhanced HTML template with improved responsive design
# Save the enhanced HTML template with improved responsive design

# Save the enhanced HTML template with the new radio buttons
# Save the enhanced HTML template with the matching radio button colors
# Save the enhanced HTML template with the new submit button
# Save the enhanced HTML template with the hidden progress bar initially
# Save the enhanced HTML template with the new animated background
# Save the enhanced HTML template with the new canvas background
# Save the enhanced HTML template with the interactive canvas background and centered content
# Save the enhanced HTML template with the starry background
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bangla Fake News Detector</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap" rel="stylesheet">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }

        html, body { width: 100vw; height: 100vh; display: flex; justify-content: center; align-items: center; overflow: hidden; }

        body {
            background: radial-gradient(ellipse at bottom, #0d1d31 0%, #0c0d13 100%);
        }

        .container {
            background: rgba(22, 33, 62, 0.9);
            padding: 40px;
            border-radius: 12px;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
            width: 100%;
            max-width: 500px;
            text-align: center;
            position: relative;
            z-index: 1;
            color: #ffffff; /* Set the text color to be visible against the background */
        }

        h1 {
            color: #00d4ff;
            margin-bottom: 20px;
            font-size: 2em;
            animation: fadeInDown 1s ease-out;
        }

        .radio-inputs {
            position: relative;
            display: flex;
            flex-wrap: wrap;
            border-radius: 0.5rem;
            background-color: #EEE;
            box-sizing: border-box;
            box-shadow: 0 0 0px 1px rgba(0, 0, 0, 0.06);
            padding: 0.25rem;
            width: 100%;
            max-width: 300px;
            margin: 0 auto 20px auto;
            font-size: 14px;
        }

        .radio-inputs .radio {
            flex: 1 1 auto;
            text-align: center;
        }

        .radio-inputs .radio input {
            display: none;
        }

        .radio-inputs .radio .name {
            display: flex;
            cursor: pointer;
            align-items: center;
            justify-content: center;
            border-radius: 0.5rem;
            border: none;
            padding: .5rem 0;
            color: rgba(51, 65, 85, 1);
            transition: all .15s ease-in-out;
        }

        .radio-inputs .radio input:checked + .name {
            background: linear-gradient(135deg, #1a1a2e, #16213e);
            color: #fff;
            font-weight: 600;
        }

        label {
            display: block;
            font-weight: 500;
            margin-bottom: 10px;
            text-align: left;
            animation: fadeInUp 1s ease-out;
        }

        textarea {
            width: 100%;
            padding: 12px;
            margin: 10px 0;
            border: none;
            border-radius: 8px;
            box-sizing: border-box;
            font-size: 16px;
            background: rgba(0, 0, 0, 0.2);
            color: #ffffff;
            z-index: 2;
            position: relative;
        }

        .button-89 {
            --b: 3px;   /* border thickness */
            --s: .45em; /* size of the corner */
            --color: #00d4ff; /* Match with the color scheme */

            width: 100%;
            padding: calc(.5em + var(--s)) calc(.9em + var(--s));
            color: var(--color);
            --_p: var(--s);
            background:
              conic-gradient(from 90deg at var(--b) var(--b),#0000 90deg,var(--color) 0)
              var(--_p) var(--_p)/calc(100% - var(--b) - 2*var(--_p)) calc(100% - var(--b) - 2*var(--_p));
            transition: .3s linear, color 0s, background-color 0s;
            outline: var(--b) solid #0000;
            outline-offset: .6em;
            font-size: 16px;
            border: 0;
            user-select: none;
            -webkit-user-select: none;
            touch-action: manipulation;
            margin-top: 20px;
        }

        .button-89:hover,
        .button-89:focus-visible{
            --_p: 0px;
            outline-color: var(--color);
            outline-offset: .05em;
        }

        .button-89:active {
            background: var(--color);
            color: #fff;
        }

        .result {
            margin-top: 20px;
            padding: 15px;
            border-radius: 8px;
            font-size: 1.2em;
            display: none;
            z-index: 2;
            position: relative;
            animation: slideIn 0.5s ease-out;
        }

        .result.authentic {
            background-color: rgba(40, 167, 69, 0.9);
            color: #fff;
        }

        .result.fake {
            background-color: rgba(220, 53, 69, 0.9);
            color: #fff;
        }

        .loader {
            width: 120px;
            height: 22px;
            border-radius: 40px;
            color: #514b82;
            border: 2px solid;
            position: relative;
            overflow: hidden;
            margin: 20px auto;
            display: none;
        }
        .loader::before {
            content: "";
            position: absolute;
            margin: 2px;
            width: 14px;
            top: 0;
            bottom: 0;
            left: -20px;
            border-radius: inherit;
            background: currentColor;
            box-shadow: -10px 0 12px 3px currentColor;
            clip-path: polygon(0 5%, 100% 0, 100% 100%, 0 95%, -30px 50%);
            animation: l14 1s infinite linear;
        }
        @keyframes l14 {
            100% { left: calc(100% + 20px) }
        }

        .progress-container {
            width: 100%;
            background-color: #e0e0e0;
            border-radius: 25px;
            margin-top: 20px;
            overflow: hidden;
            z-index: 2;
            display: none; /* Initially hide the progress container */
        }

        .progress-bar {
            height: 20px;
            width: 0;
            background-color: #00d4ff;
            border-radius: 25px;
            transition: width 0.5s;
            z-index: 2;
        }

        @keyframes fadeInDown {
            from {
                opacity: 0;
                transform: translateY(-20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateX(-20px);
            }
            to {
                opacity: 1;
                transform: translateX(0);
            }
        }

        /* Starry Background Styles */
        .stars {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 120%;
            transform: rotate(-45deg);
        }

        .star {
            $star-count: 50;
            --star-color: #ffffff;
            --star-tail-length: 6em;
            --star-tail-height: 2px;
            --star-width: calc(var(--star-tail-length) / 6);
            --fall-duration: 9s;
            --tail-fade-duration: var(--fall-duration);

            position: absolute;
            top: var(--top-offset);
            left: 0;
            width: var(--star-tail-length);
            height: var(--star-tail-height);
            color: var(--star-color);
            background: linear-gradient(45deg, currentColor, transparent);
            border-radius: 50%;
            filter: drop-shadow(0 0 6px currentColor);
            transform: translate3d(104em, 0, 0);
            animation: fall var(--fall-duration) var(--fall-delay) linear infinite, tail-fade var(--tail-fade-duration) var(--fall-delay) ease-out infinite;

            @for $i from 1 through $star-count {
                &:nth-child(#{$i}) {
                    --star-tail-length: #{random(50)}em;
                    --top-offset: #{random(100)}vh;
                    --fall-duration: #{random(6) + 6}s;
                    --fall-delay: #{random(10)}s;
                }
            }

            &::before, &::after {
                position: absolute;
                content: '';
                top: 0;
                left: calc(var(--star-width) / -2);
                width: var(--star-width);
                height: 100%;
                background: linear-gradient(45deg, transparent, currentColor, transparent);
                border-radius: inherit;
                animation: blink 2s linear infinite;
            }

            &::before {
                transform: rotate(45deg);
            }

            &::after {
                transform: rotate(-45deg);
            }
        }

        @keyframes fall {
            to {
                transform: translate3d(-30em, 0, 0);
            }
        }

        @keyframes tail-fade {
            0%, 50% {
                width: var(--star-tail-length);
                opacity: 1;
            }

            70%, 80% {
                width: 0;
                opacity: 0.4;
            }

            100% {
                width: 0;
                opacity: 0;
            }
        }

        @keyframes blink {
            50% {
                opacity: 0.6;
            }
        }
    </style>
</head>
<body>
    <div class="stars">
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
        <div class="star"></div>
    </div>

    <div class="container">
        <h1>Bangla Fake News Detector</h1>
        <form id="newsForm">
            <div class="radio-inputs">
                <label class="radio">
                    <input type="radio" name="data_type" value="text" checked>
                    <span class="name">News Text</span>
                </label>
                <label class="radio">
                    <input type="radio" name="data_type" value="url">
                    <span class="name">News URL</span>
                </label>
            </div>
            <label for="news">Enter news text or URL:</label>
            <textarea id="news" name="news" rows="4"></textarea>
            <button class="button-89" role="button">Submit</button>
        </form>
        <div class="loader" id="loader"></div>
        <div id="result" class="result"></div>
        <div class="progress-container" id="progressContainer">
            <div id="progressBar" class="progress-bar"></div>
        </div>
    </div>
    <script>
        document.addEventListener('DOMContentLoaded', function () {
            const form = document.getElementById('newsForm');
            const resultDiv = document.getElementById('result');
            const loader = document.getElementById('loader');
            const progressBar = document.getElementById('progressBar');
            const progressContainer = document.getElementById('progressContainer');

            form.onsubmit = async function (e) {
                e.preventDefault();
                resultDiv.style.display = 'none';
                loader.style.display = 'block';
                progressBar.style.width = '0';
                progressContainer.style.display = 'block'; // Show the progress bar when the form is submitted
                const formData = new FormData(form);
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
                const data = await response.json();
                loader.style.display = 'none';
                resultDiv.style.display = 'block';
                resultDiv.innerHTML = `Prediction: ${data.prediction} (${data.percentage}%)`;
                if (data.prediction === 'Authentic News') {
                    resultDiv.className = 'result authentic';
                    progressBar.style.backgroundColor = '#28a745';
                } else {
                    resultDiv.className = 'result fake';
                    progressBar.style.backgroundColor = '#dc3545';
                }
                progressBar.style.width = `${data.percentage}%`;
            };
        });
    </script>
</body>
</html>
"""


with open('templates/index.html', 'w') as f:
    f.write(html_template)

# Load the model and tokenizer
model_path = '/content/drive/MyDrive/Capstone_shared/bangla_fake_news_model.h5'
tokenizer_path = '/content/drive/MyDrive/Capstone_shared/tokenizer.pkl'
# Load the model using Keras' load_model method
model = tf.keras.models.load_model(model_path)

# Load the tokenizer
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Preprocessing function
stop_words = set(stopwords.words('bengali'))
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Function to predict news authenticity
def predict_news(news):
    news = preprocess(news)
    seq = tokenizer.texts_to_sequences([news])
    padded = pad_sequences(seq, padding='post', maxlen=200)
    prediction = model.predict(padded)[0][0]
    percentage = round(prediction * 100, 2) if prediction > 0.5 else round((1 - prediction) * 100, 2)
    return ("Fake News" if prediction > 0.5 else "Authentic News"), percentage

# Function to get news content from a URL using Selenium
def get_news_content_selenium(url):
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=chrome_options)

        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        content = driver.find_element(By.TAG_NAME, 'body').text
        driver.quit()
        return content
    except Exception as e:
        return f"Error: Could not retrieve content. Exception: {e}"

# Function to predict news authenticity from a URL
def predict_news_url_selenium(url):
    if isinstance(url, str):
        content = get_news_content_selenium(url)
        if content.startswith("Error"):
            return content, 0
        return predict_news(content)
    else:
        return "Error: URL is not a string.", 0

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if request.method == 'POST':
            data_type = request.form.get('data_type')
            if data_type == 'text':
                news = request.form.get('news')
                prediction, percentage = predict_news(news)
            elif data_type == 'url':
                url = request.form.get('news')
                prediction, percentage = predict_news_url_selenium(url)
            return jsonify({'prediction': prediction, 'percentage': percentage})
    except Exception as e:
        return str(e), 400

# Set ngrok auth token and connect
ngrok.set_auth_token("2iq8btcKUJUwcpIvds0cZ3VcqNh_hZoSVsfv93hx5NFpAGxq")  # Ensure to set your ngrok auth token here
public_url = ngrok.connect(5000, bind_tls=True)
print('Public URL:', public_url)

# Run the Flask app
app.run(port=5000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Public URL: NgrokTunnel: "https://f2f5-34-125-44-120.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:10:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:11:00] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step


INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:13:25] "POST /predict HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:16:38] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:16:58] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


INFO:werkzeug:127.0.0.1 - - [28/Aug/2024 07:17:49] "POST /predict HTTP/1.1" 200 -
